In [9]:
# This is the FriendshipGoals CNN
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'1.14.0'

In [10]:
# USING GOOGLE IMAGES FOR A DATASET

In [11]:
# Data Preprocessing
# need to apply transformations to test set to correct overfitting
# I don't know if Sample_Data is our actual test set tbh there's only 3 images
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('Dataset/Training_Data',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 124 images belonging to 3 classes.


In [12]:
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
# very similar to how in the ANN we used fit_transform on the training set, but just
# fit on the test set because we didn't want any information leakage from the training set.
# Don't want to mess with the images too much so they're intact like they would be in production
test_set = test_datagen.flow_from_directory('Dataset/Testing_Data',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 314 images belonging to 1 classes.


In [13]:
# building the model
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, 
                               kernel_size=3, 
                               input_shape=[64,64,3], 
                               activation='relu'))
# filters = number of feature detectors
# input shape = (64, 64, 3) because we resized pictures to 64x64, and they're RGB
# kernel = size of feature detectors (e.g. 3 = 3x3 square)
# activation = activation function

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, 
                               kernel_size=3, 
                               activation='relu'))
# dont need input shape
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [14]:
# flattening
cnn.add(tf.keras.layers.Flatten())

# full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
# output layer
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax')) # 3 classes

In [15]:
# training the CNN
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
cnn.fit(x=training_set, epochs=25) # validation_data='test_set'

Epoch 1/25
4/4 [==============================] - 3s 765ms/step - loss: 1.2554 - acc: 0.4274
Epoch 2/25
4/4 [==============================] - 3s 685ms/step - loss: 1.1313 - acc: 0.3387
Epoch 3/25
4/4 [==============================] - 3s 685ms/step - loss: 1.0850 - acc: 0.3226
Epoch 4/25
4/4 [==============================] - 3s 681ms/step - loss: 1.0597 - acc: 0.4355
Epoch 5/25
4/4 [==============================] - 3s 683ms/step - loss: 1.0301 - acc: 0.6532
Epoch 6/25
4/4 [==============================] - 3s 689ms/step - loss: 0.9781 - acc: 0.5968
Epoch 7/25
4/4 [==============================] - 3s 696ms/step - loss: 0.9363 - acc: 0.6210
Epoch 8/25
4/4 [==============================] - 3s 681ms/step - loss: 0.8608 - acc: 0.5968
Epoch 9/25
4/4 [==============================] - 3s 684ms/step - loss: 0.8433 - acc: 0.6855
Epoch 10/25
4/4 [==============================] - 3s 679ms/step - loss: 0.7370 - acc: 0.6774
Epoch 11/25
4/4 [==============================] - 3s 701ms/step - lo

In [17]:
# Making a single prediction
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('00000042.jpg', target_size=(64,64))

# the predict method expects a 2D array
test_image = image.img_to_array(test_image)

# the training was done on BATCHES of images, we need to add a dimension corresponding to the
# batch. We want the axis we're adding to be the first dimension (axis=0)
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image)
training_set.class_indices

if result[0][0] == 0:  #first [0] is getting access to the batch, second is first and 
    prediction= 'Adults' #only element of the batch which is the prediction of the single image
elif result[0][0] == 1:
    prediction= 'Teenagers'
else:
    prediction = 'Toddler'
    
print(prediction)


    

Teenagers


In [18]:
# putting it into a method
def pred(path):
    test_image = image.load_img(path, target_size=(64,64))

    test_image = image.img_to_array(test_image)

    test_image = np.expand_dims(test_image, axis=0)

    result = cnn.predict(test_image)
    training_set.class_indices

    if result[0][0] == 0: 
        prediction= 'Adults'
    elif result[0][0] == 1:
        prediction= 'Teenagers'
    else:
        prediction = 'Toddlers'
        
    return prediction

In [19]:
# Now we have to read the csv file, and loop through all the filenames while filling the 
# empty second column with our predictions
from csv import reader, writer

# open file in read mode
with open('Dataset/Test.csv', 'r') as csvinput:
    with open('Dataset/bmcshaneSubmission.csv', 'w') as csvoutput:
        csv_reader = reader(csvinput)
        csv_writer = writer(csvoutput, lineterminator='\n')
        
        answers = []
        row = next(csv_reader)
        row.append('Category')
        answers.append(row)
        
        
        for row in csv_reader:
            guess = pred('Dataset/Testing_Data/Data/' + row[0])
            row.append(guess)
            
            answers.append(row)
        
        csv_writer.writerows(answers)
